# On-line news popularity
                                        Database with PostgreSQL
Report by Carlos Neves for [Thinkful](https://www.thinkful.com/)
### Aim:

The aim of this report is to illustrate the use of Structured Query Language ([SQL](https://en.wikipedia.org/wiki/SQL)) for working with databases with the [PostgreSQL](https://www.postgresql.org/) implementation for Python. Specifically, the [Online News Popularity](http://archive.ics.uci.edu/ml/datasets/Online+News+Popularity) dataset from the University of California, Irvine (UCI) [Machine Learning repository](http://archive.ics.uci.edu/ml/index.php) will be saved into a database and some simple statistics extracted. The data was generated by K. Fernandes, P. Vinagre and P. Cortez for their 2015 study [*A Proactive Intelligent Decision Support System for Predicting the Popularity of Online News*](https://link.springer.com/chapter/10.1007/978-3-319-23485-4_53). 

In [1]:
# Import required packages
import psycopg2 as psql
import pandas as pd
import numpy as np
import datetime as dt

# Magics
%matplotlib inline

# Load data 
news = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')

news.columns = news.columns.str.strip()
news.columns.values[news.columns == 'self_reference_avg_sharess'] = 'self_reference_avg_shares'

# Pre-process the data
channels = [col for col in news.columns if 'data_channel' in col]
news['channel'] = 'other'
for channel_id in channels: 
    news.loc[news[channel_id] == 1,'channel'] = channel_id.replace('data_channel_is_', '')

weekdays = [col for col in news.columns if 'weekday' in col]
news['weekday'] = np.nan
for weekday_id in weekdays: 
    news.loc[news[weekday_id] == 1,'weekday'] = weekday_id.replace('weekday_is_', '')

dataset_date = dt.datetime.strptime('8 Jan 2015', '%d %b %Y')
news['publication_date'] = news['timedelta'].apply(lambda x: dataset_date - dt.timedelta(days=x))

news['article_name'] = news['url'].replace(r'.*/(.*)/', '\\1', regex=True)
news['article_name'] = news['article_name'].replace(r'-', ' ', regex=True)

## Databases 

A [relational database](https://en.wikipedia.org/wiki/Relational_database) is a powerful tool for formatting and organizing data. Unlike text files, a remote database can be distributed (across many machines), resulting in a higher capacity, simultaneous access of multiple users to the same data version and uniform storage of data from multiple sources in a single system. 
 
A database typically consists of a series of *tables*, each with its own defined organization. A table is organized first into *columns* where each column has a name and a data type (and other configurable requirements), and then into *records* (rows) which must conform to the table schema. The *schema* is a formal statement about how the tables for a database are set up. As such, a database contains one or more schemas, which in turn contain tables, which are organised by columns. Schemas may also contain more advanced objects, including *data types*, *functions* and *operators*. Even though each schema may have many tables, because each table is organised according to one schema only, this term can also refer to a particular table's configuration of columns.  Schemas can change over time according to the developer's needs, but there is a cost to modifying a schema, as it requires migrating (systematically updating) existing data to support the schema changes. 

### SQL
To analyse data, it first needs to be accessed. If stored in a database, this access is accomplished through requests for specific parts of the database, known as  *queries*. SQL is the common language for writing these queries and can be used to create, retrieve, update and delete database records. There are many different forms of SQL, each implemented by the particular database vendor who wrote the database. While there are meaningful minor differences between each implementation (PostgreSQL, MySQL, SQLite, *etc.*), the overarching concepts are consistent across database systems. A SQL query is made using *statements*, the code that defines what the database should do (SELECT, DELETE, UPDATE, *etc*). These statements are made of various *clauses*, which define certain instructions in a statement (WHERE, ORDERBY , *etc*). The building blocks of all queries are the *keywords*, that is, the specific SQL lexicon and reserved words.

#### SQL in Python

The [psycopg2](http://initd.org/psycopg/) package implements PostgreSQL in Python, allowing for a easy interaction with a database. To do this, the [connect function](http://initd.org/psycopg/docs/connection.html) is used create a connection object and initiate a database session. Databases often have restricted access and require authentication to connect, so a user, host and password arguments may be required. Once there is an established connection, a pointer needs to be created using the connection method [cursor](http://initd.org/psycopg/docs/cursor.html). This cursor can then interact with the database and execute any valid SQL query using the [execute](http://initd.org/psycopg/docs/cursor.html?highlight=execute#cursor.execute) method. A new database can be created using the [CREATE DATABASE](https://www.postgresql.org/docs/9.0/sql-createdatabase.html) clause. 

In [2]:
# Connect to the server
connection = psql.connect(dbname='postgres',
      user='carlosneves', host='localhost',
      password='thinkful')
connection.autocommit = True

cursor = connection.cursor()

# Create database
cursor.execute('''DROP DATABASE IF EXISTS onlinenews''')
cursor.execute('''CREATE DATABASE onlinenews
                  WITH 
                      OWNER = carlosneves
                      ENCODING = 'UTF8'
                      TABLESPACE = pg_default
                      LC_COLLATE = 'en_GB.UTF-8'
                      LC_CTYPE = 'en_GB.UTF-8'
                      CONNECTION LIMIT = -1;
               ''')

cursor.close()
connection.close()

# Reconnect to new database
connection = psql.connect(dbname='onlinenews',
      user='carlosneves', host='localhost',
      password='thinkful')

cursor = connection.cursor()

# Print connection properties
print (connection.get_dsn_parameters(),"\n")

{'user': 'carlosneves', 'dbname': 'onlinenews', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



Note that some parts of the previous SQL queries are written in upper case, while other are written in lower case. **Even though this is not mandatory**, it is a convention that keywords (SQL statements, data types, *etc.*) are written in upper-case and identifiers (table names, operations, *etc.*) are written in lower-case. This convention makes SQL easier to read, which was crucial as this language pre-dates the modern text editors with specific language highlight.

In [3]:
# Helper functions
types_code = {'text': str, 
             'integer': int, 
             'real': float, 
             'boolean': bool,
             'date': dt.date}

def sql_query(query, cursor=cursor):
    '''
    Function to query the database.
    Takes as arguments:
        query: A string matching an SQL query.
        cursor: The PostgreSQL pointer being used to interact with the database.
    '''
    
    # Execute query
    cursor.execute(query)
    
    # Return query as data frame
    return(pd.DataFrame(cursor.fetchall(), columns =[desc[0] for desc in cursor.description]))

def add2sql(data, table, types_code = types_code, cursor=cursor, caps = False, show = False):
    '''
    Function to add values from a pandas data frame to a database table.
    Takes as arguments:
        data: The data frame containing the variables to add to the table.
        table: The table from the database to add the data to.
        types_code: Dictionary mapping between the Python and SQL data type.
        cursor: The PostgreSQL pointer being used to interact with the database.
        caps: Boolean parameter defining if all column names from the database table should be converted to upper case to match the data frame.
        
    '''
    
    # Query the data base for the names of the table columns
    col_query = '''SELECT column_name, data_type 
                   FROM information_schema.columns 
                   WHERE table_name = '%s';''' % (table)
    cursor.execute(col_query)
    res = cursor.fetchall()
    
    # Extract the columns names and types from the database
    cols  = [r[0] for r in res]
    if caps: cols[1:] = list(map(str.upper, cols[1:]))
    types  = [types_code[r[1]] for r in res]
    
    # Subset the data frame to match the database table
    data = data.reset_index()[['index'] + cols[1:]]

    insert_query = 'INSERT INTO %s (%s)\nVALUES (%s)' % (table, ', '.join(cols), (', %s'* len(cols))[2:])
    
    print('Query:\n' + insert_query)
    
    # Loop through all records and add them to the database table with the correct data type
    for row in range(data.shape[0]):
        cursor.execute(insert_query, 
                    [types[col](data.iloc[row, col]) if not types[col] is dt.date else str(data.iloc[row,col].date()) for col in range(data.shape[1])]
                  )
        
    # Return a query with the changed table    
    if show: return(sql_query('SELECT * FROM %s' % (table)))


## Create tables

A new table can be added to a database using the [CREATE TABLE](https://www.postgresql.org/docs/9.0/sql-createtable.html) clause: These keywords are followed by the name of the table to be created and the statements to configure individual columns in brackets, defining at least the columns' name and [data type](https://www.postgresql.org/docs/current/datatype.html). Note that [constrains](https://www.postgresql.org/docs/current/ddl-constraints.html) can be defined for each column using keywords such as CHECK, PRIMARY KEY and NOT NULL, for example, to force the values in a column to be unique or prohibiting null values.

In [4]:
# Query to create news table
cursor.execute('''CREATE TABLE news (
                      id INT PRIMARY KEY,
                      url TEXT,
                      shares INTEGER CHECK (shares >=0),
                      article_name TEXT
                  );
               ''')
# Query data to the news table
add2sql(news, 'news', show = True)

Query:
INSERT INTO news (id, url, shares, article_name)
VALUES (%s, %s, %s, %s)


id                                                url  shares  \
0          0  http://mashable.com/2013/01/07/amazon-instant-...     593   
1          1  http://mashable.com/2013/01/07/ap-samsung-spon...     711   
2          2  http://mashable.com/2013/01/07/apple-40-billio...    1500   
3          3  http://mashable.com/2013/01/07/astronaut-notre...    1200   
4          4   http://mashable.com/2013/01/07/att-u-verse-apps/     505   
5          5   http://mashable.com/2013/01/07/beewi-smart-toys/     855   
6          6  http://mashable.com/2013/01/07/bodymedia-armba...     556   
7          7   http://mashable.com/2013/01/07/canon-poweshot-n/     891   
8          8  http://mashable.com/2013/01/07/car-of-the-futu...    3600   
9          9  http://mashable.com/2013/01/07/chuck-hagel-web...     710   
10        10  http://mashable.com/2013/01/07/cosmic-events-d...    2200   
11        11   http://mashable.com/2013/01/07/crayon-creatures/    1900   
12        12      http://mashable.com/2013/01/07/creature-cups/     823   
13        13          http://mashable.com/2013/01/07/dad-jokes/   10000   
14        14  http://mashable.com/2013/01/07/downton-abbey-t...     761   
15        15  http://mashable.com/2013/01/07/earth-size-plan...    1600   
16        16          http://mashable.com/2013/01/07/echo-game/   13600   
17        17  http://mashable.com/2013/01/07/entrepreneur-tr...    3100   
18        18  http://mashable.com/2013/01/07/facebook-sick-app/    5700   
19        19  http://mashable.com/2013/01/07/felt-audio-puls...   17100   
20        20       http://mashable.com/2013/01/07/ford-glympse/    2800   
21        21   http://mashable.com/2013/01/07/ftc-google-leaks/     598   
22        22  http://mashable.com/2013/01/07/fujifilm-50x-su...     445   
23        23  http://mashable.com/2013/01/07/hillary-clinton...    1500   
24        24             http://mashable.com/2013/01/07/htc-q1/     852   
25        25  http://mashable.com/2013/01/07/huawei-ascend-m...     783   
26        26  http://mashable.com/2013/01/07/iheartradio-app...    1500   
27        27  http://mashable.com/2013/01/07/intel-awesome-l...    1800   
28        28  http://mashable.com/2013/01/07/isp02-iphone-ta...     462   
29        29     http://mashable.com/2013/01/07/jobs-contently/     425   
...      ...                                                ...     ...   
39614  39614  http://mashable.com/2014/12/26/the-interview-i...    1400   
39615  39615  http://mashable.com/2014/12/26/toothpaste-fluo...    5700   
39616  39616  http://mashable.com/2014/12/26/top-photography...    2100   
39617  39617  http://mashable.com/2014/12/26/tor-attacked-li...     691   
39618  39618  http://mashable.com/2014/12/26/tsa-gift-wrap-r...    1400   
39619  39619  http://mashable.com/2014/12/26/tsunami-10-year...    1200   
39620  39620  http://mashable.com/2014/12/26/turkey-teen-ins...    2400   
39621  39621  http://mashable.com/2014/12/26/ukraine-prisone...   24300   
39622  39622  http://mashable.com/2014/12/26/wacky-kickstart...    2900   
39623  39623        http://mashable.com/2014/12/26/what-is-tor/     947   
39624  39624  http://mashable.com/2014/12/27/air-asia-flight...    3200   
39625  39625  http://mashable.com/2014/12/27/als-ice-bucket-...    1400   
39626  39626  http://mashable.com/2014/12/27/christmas-tree-...    1100   
39627  39627  http://mashable.com/2014/12/27/extremist-leade...    1200   
39628  39628  http://mashable.com/2014/12/27/facebook-year-i...    1000   
39629  39629  http://mashable.com/2014/12/27/frozen-doll-prank/    2400   
39630  39630  http://mashable.com/2014/12/27/high-school-i-c...    1500   
39631  39631  http://mashable.com/2014/12/27/japan-elderly-d...     914   
39632  39632  http://mashable.com/2014/12/27/las-vegas-new-y...    1700   
39633  39633  http://mashable.com/2014/12/27/lbj-adviser-sla...    1500   
39634  39634  http://mashable.com/2014/12/27/music-grid-outf...    1000   
39635  39635   http://mashable.com/

In [5]:
# Query to create words table
cursor.execute('''CREATE TABLE words (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      n_tokens_title INTEGER CHECK (n_tokens_title >=0),
                      n_tokens_content INTEGER CHECK (n_tokens_content >=0),
                      n_non_stop_words INTEGER CHECK (n_non_stop_words >=0),
                      n_unique_tokens REAL CHECK (n_unique_tokens >=0),
                      n_non_stop_unique_tokens REAL CHECK (n_non_stop_unique_tokens >=0),
                      average_token_length REAL CHECK (average_token_length >=0)
                  );
               ''')
# Query data to the words table
add2sql(news, 'words')

Query:
INSERT INTO words (id, n_tokens_title, n_tokens_content, n_non_stop_words, n_unique_tokens, n_non_stop_unique_tokens, average_token_length)
VALUES (%s, %s, %s, %s, %s, %s, %s)


In [6]:
# Query to create channel table
cursor.execute('''CREATE TABLE channel (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      channel TEXT,
                      data_channel_is_lifestyle BOOLEAN,
                      data_channel_is_entertainment BOOLEAN,
                      data_channel_is_bus BOOLEAN,
                      data_channel_is_socmed BOOLEAN,
                      data_channel_is_tech BOOLEAN,
                      data_channel_is_world BOOLEAN
                  );
               ''')
# Query data to the channel table
add2sql(news, 'channel', show = True)

Query:
INSERT INTO channel (id, channel, data_channel_is_lifestyle, data_channel_is_entertainment, data_channel_is_bus, data_channel_is_socmed, data_channel_is_tech, data_channel_is_world)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)


id        channel  data_channel_is_lifestyle  \
0          0  entertainment                      False   
1          1            bus                      False   
2          2            bus                      False   
3          3  entertainment                      False   
4          4           tech                      False   
5          5           tech                      False   
6          6      lifestyle                       True   
7          7           tech                      False   
8          8           tech                      False   
9          9          world                      False   
10        10          world                      False   
11        11      lifestyle                       True   
12        12          other                      False   
13        13          other                      False   
14        14          other                      False   
15        15          world                      False   
16        16          other                      False   
17        17            bus                      False   
18        18      lifestyle                       True   
19        19           tech                      False   
20        20           tech                      False   
21        21          world                      False   
22        22           tech                      False   
23        23          world                      False   
24        24            bus                      False   
25        25           tech                      False   
26        26           tech                      False   
27        27           tech                      False   
28        28      lifestyle                       True   
29        29            bus                      False   
...      ...            ...                        ...   
39614  39614          world                      False   
39615  39615          world                      False   
39616  39616          other                      False   
39617  39617          world                      False   
39618  39618          world                      False   
39619  39619          world                      False   
39620  39620          world                      False   
39621  39621          other                      False   
39622  39622          world                      False   
39623  39623           tech                      False   
39624  39624           tech                      False   
39625  39625           tech                      False   
39626  39626           tech                      False   
39627  39627          world                      False   
39628  39628          world                      False   
39629  39629  entertainment                      False   
39630  39630          world                      False   
39631  39631          world                      False   
39632  39632          world                      False   
39633  39633          world                      False   
39634  39634          other                      False   
39635  39635            bus                      False   
39636  39636      lifestyle                       True   
39637  39637  entertainment                      False   
39638  39638            bus                      False   
39639  39639           tech                      False   
39640  39640         socmed                      False   
39641  39641          other                      False   
39642  39642          world                      False   
39643  39643  entertainment                      False   

       data_channel_is_entertainment  data_channel_is_bus  \
0                               True                False   
1                              False                 True   
2                              False                 True   
3                               True                False   
4                              False                False   
5                              False           

In [7]:
# Query to create reference table
cursor.execute('''CREATE TABLE reference (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      num_hrefs INTEGER CHECK (num_hrefs >=0),
                      num_self_hrefs INTEGER CHECK (num_self_hrefs >=0),
                      num_imgs INTEGER CHECK (num_imgs >=0),
                      num_videos INTEGER CHECK (num_videos >=0),
                      self_reference_min_shares INTEGER CHECK (self_reference_min_shares >=0),
                      self_reference_max_shares INTEGER CHECK (self_reference_max_shares >=0),
                      self_reference_avg_shares INTEGER CHECK (self_reference_avg_shares >=0)
                  );
               ''')
# Query data to the reference table
add2sql(news, 'reference')

Query:
INSERT INTO reference (id, num_hrefs, num_self_hrefs, num_imgs, num_videos, self_reference_min_shares, self_reference_max_shares, self_reference_avg_shares)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)


In [8]:
# Query to create time table
cursor.execute('''CREATE TABLE time (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      timedelta INTEGER,
                      publication_date DATE,
                      weekday_is_monday BOOLEAN,
                      weekday_is_tuesday BOOLEAN,
                      weekday_is_wednesday BOOLEAN,
                      weekday_is_thursday BOOLEAN,
                      weekday_is_friday BOOLEAN,
                      weekday_is_saturday BOOLEAN,
                      weekday_is_sunday BOOLEAN,
                      is_weekend BOOLEAN,
                      weekday TEXT
                    );
               ''')
# Query data to the time table
add2sql(news, 'time', show = True)

Query:
INSERT INTO time (id, timedelta, publication_date, weekday_is_monday, weekday_is_tuesday, weekday_is_wednesday, weekday_is_thursday, weekday_is_friday, weekday_is_saturday, weekday_is_sunday, is_weekend, weekday)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


id  timedelta publication_date  weekday_is_monday  \
0          0        731       2013-01-07               True   
1          1        731       2013-01-07               True   
2          2        731       2013-01-07               True   
3          3        731       2013-01-07               True   
4          4        731       2013-01-07               True   
5          5        731       2013-01-07               True   
6          6        731       2013-01-07               True   
7          7        731       2013-01-07               True   
8          8        731       2013-01-07               True   
9          9        731       2013-01-07               True   
10        10        731       2013-01-07               True   
11        11        731       2013-01-07               True   
12        12        731       2013-01-07               True   
13        13        731       2013-01-07               True   
14        14        731       2013-01-07               True   
15        15        731       2013-01-07               True   
16        16        731       2013-01-07               True   
17        17        731       2013-01-07               True   
18        18        731       2013-01-07               True   
19        19        731       2013-01-07               True   
20        20        731       2013-01-07               True   
21        21        731       2013-01-07               True   
22        22        731       2013-01-07               True   
23        23        731       2013-01-07               True   
24        24        731       2013-01-07               True   
25        25        731       2013-01-07               True   
26        26        731       2013-01-07               True   
27        27        731       2013-01-07               True   
28        28        731       2013-01-07               True   
29        29        731       2013-01-07               True   
...      ...        ...              ...                ...   
39614  39614          9       2014-12-30              False   
39615  39615          9       2014-12-30              False   
39616  39616          9       2014-12-30              False   
39617  39617          9       2014-12-30              False   
39618  39618          9       2014-12-30              False   
39619  39619          9       2014-12-30              False   
39620  39620          9       2014-12-30              False   
39621  39621          9       2014-12-30              False   
39622  39622          9       2014-12-30              False   
39623  39623          9       2014-12-30              False   
39624  39624          9       2014-12-30              False   
39625  39625          9       2014-12-30              False   
39626  39626          9       2014-12-30              False   
39627  39627          9       2014-12-30              False   
39628  39628          9       2014-12-30              False   
39629  39629          9       2014-12-30              False   
39630  39630          9       2014-12-30              False   
39631  39631          9       2014-12-30              False   
39632  39632          9       2014-12-30              False   
39633  39633          9       2014-12-30              False   
39634  39634          9       2014-12-30              False   
39635  39635          9       2014-12-30              False   
39636  39636          8       2014-12-31              False   
39637  39637          8       2014-12-31              False   
39638  39638          8       2014-12-31              False   
39639  39639          8       2014-12-31              False   
39640  39640          8       2014-12-31              False   
39641  39641          8       2014-12-31              False   
39642  39642          8       2014-12-31              False   
39643  39643          8       2014-12-31              False   

       weekday_is_tuesday  weekday_is_wednesday  weekday_is_thursday  \
0                   False      

In [9]:
# Query to create keywords table
cursor.execute('''CREATE TABLE keywords (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      num_keywords INTEGER CHECK (num_keywords >=0),
                      kw_min_min INTEGER,
                      kw_max_min INTEGER CHECK (kw_max_min >=0),
                      kw_avg_min REAL,
                      kw_min_max INTEGER,
                      kw_max_max INTEGER CHECK (kw_max_min >=0),
                      kw_avg_max REAL,
                      kw_min_avg REAL,
                      kw_max_avg REAL,
                      kw_avg_avg REAL
                  );
               ''')
# Query data to the keywords table
add2sql(news, 'keywords')

Query:
INSERT INTO keywords (id, num_keywords, kw_min_min, kw_max_min, kw_avg_min, kw_min_max, kw_max_max, kw_avg_max, kw_min_avg, kw_max_avg, kw_avg_avg)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


In [10]:
# Query to create lda table
cursor.execute('''CREATE TABLE lda (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      LDA_00 REAL,
                      LDA_01 REAL,
                      LDA_02 REAL,
                      LDA_03 REAL,
                      LDA_04 REAL
                   );
               ''')
# Query data to the lda table
add2sql(news, 'lda', caps = True)

Query:
INSERT INTO lda (id, LDA_00, LDA_01, LDA_02, LDA_03, LDA_04)
VALUES (%s, %s, %s, %s, %s, %s)


In [11]:
# Query to create sentiment table
cursor.execute('''CREATE TABLE sentiment (
                      id INT PRIMARY KEY REFERENCES news(id) ON DELETE CASCADE,
                      title_subjectivity   REAL,
                      title_sentiment_polarity  REAL,
                      abs_title_subjectivity  REAL,
                      abs_title_sentiment_polarity  REAL,
                      global_subjectivity  REAL,
                      global_sentiment_polarity  REAL,
                      global_rate_positive_words  REAL,
                      global_rate_negative_words  REAL,
                      rate_positive_words  REAL,
                      avg_positive_polarity  REAL,
                      min_positive_polarity  REAL,
                      max_positive_polarity  REAL,
                      rate_negative_words  REAL,
                      avg_negative_polarity  REAL,
                      min_negative_polarity  REAL,
                      max_negative_polarity  REAL
                  );
               ''')
# Query data to the sentiment table
add2sql(news, 'sentiment')

Query:
INSERT INTO sentiment (id, title_subjectivity, title_sentiment_polarity, abs_title_subjectivity, abs_title_sentiment_polarity, global_subjectivity, global_sentiment_polarity, global_rate_positive_words, global_rate_negative_words, rate_positive_words, avg_positive_polarity, min_positive_polarity, max_positive_polarity, rate_negative_words, avg_negative_polarity, min_negative_polarity, max_negative_polarity)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)


In [12]:
# Confirm database tables
connection.commit()
sql_query('''SELECT tablename 
             FROM pg_catalog.pg_tables
             WHERE schemaname = 'public' 
          ''').T

0        1          2     3         4    5     6          7
tablename  words  channel  reference  time  keywords  lda  news  sentiment

## Selecting data

Once a connection is established, SQL can be used to extracted data from the database.
To request specific data from a database, the [SELECT](https://www.postgresql.org/docs/9.0/sql-select.html) statement is used. The keyword SELECT defines the columns containing the data while the keyword FROM defines the table where these columns are located. In addition, rows that match a specified condition can be selected with a WHERE clause, which uses [comparison operators](https://www.postgresql.org/docs/11/functions-comparison.html) (=, >, >=, *etc.*) to identify relevant records. Multiple conditions can be defined using AND, OR and BETWEEN, while the complement of a condition can be obtained using NOT. As a note, LIKE works similarly to =, but is used for pattern matching on string data.

In [13]:
# Query to return
# the columns  id, weekday_is_friday, weekday_is_saturday and weekday_is_sunday
# from the time table
# where the record values have even numbered ids between 10 and 1000 and correspond to either weekend or friday articles
sql_query('''SELECT 
                id,
                weekday_is_friday,
                weekday_is_saturday,
                weekday_is_sunday
             FROM time
             WHERE 
                (is_weekend = True OR weekday LIKE 'friday') AND
                id BETWEEN 10 AND 1000 AND NOT id % 2 = 0 
             -- By the way, this is a comment!
          ''')

id  weekday_is_friday  weekday_is_saturday  weekday_is_sunday
0   261               True                False              False
1   263               True                False              False
2   265               True                False              False
3   267               True                False              False
4   269               True                False              False
5   271               True                False              False
6   273               True                False              False
7   275               True                False              False
8   277               True                False              False
9   279               True                False              False
10  281               True                False              False
11  283               True                False              False
12  285               True                False              False
13  287               True                False              False
14  289               True                False              False
15  291               True                False              False
16  293               True                False              False
17  295               True                False              False
18  297               True                False              False
19  299               True                False              False
20  301               True                False              False
21  303               True                False              False
22  305               True                False              False
23  307               True                False              False
24  309               True                False              False
25  311              False                 True              False
26  313              False                 True              False
27  315              False                 True              False
28  317              False                 True              False
29  319              False                 True              False
..  ...                ...                  ...                ...
65  687               True                False              False
66  689               True                False              False
67  691               True                False              False
68  693               True                False              False
69  695               True                False              False
70  697               True                False              False
71  699               True                False              False
72  701               True                False              False
73  703               True                False              False
74  705               True                False              False
75  707              False                 True              False
76  709              False                 True              False
77  711              False                 True              False
78  713              False                 True              False
79  715              False                 True              False
80  717              False                 True              False
81  719              False                 True              False
82  721              False                 True              False
83  723              False                False               True
84  725              False                False               True
85  727              False                False               True
86  729              False                False               True
87  731              False                False               True
88  733              False                False               True
89  735              False                False               True
90  737              False                False               True
91  739              False                False               True
92  741              False                False     

The returned results can also be ordered by the values of one or more columns, specifying either ascending or descending order, using the a ORDER BY clause. Another important clause to keep in mind is LIMIT, which limits the number of returned results.

In [14]:
# Query to return
# the 10 article names and share values ordered in descending order
# from the news table
sql_query('''SELECT 
                article_name,
                shares
             FROM news
             ORDER BY shares DESC
             -- DESC makes set a descending order. Ascending order is the default
             LIMIT 10
          ''')

article_name  shares
0                      low cost iphone  843300
1              dove ad beauty sketches  690400
2  first 100 gilt soundcloud stitchfix  663600
3           kanye west harvard lecture  652900
4                    wealth inequality  617900
5                    roomba 880 review  441000
6            australia heatwave photos  310800
7                 blackberry 1 million  306100
8                     ibm watson brief  298400
9          ebola cdc active monitoring  284700

### Grouping data

Grouping database records is possible using the GROUP BY clause. This clause comes after the WHERE clause and before any ORDER BY clauses. On its own, grouping simply gets rid of duplicates, so all columns included in the SELECT statement must also be included in your GROUP BY statement. However, grouped data can also be aggregated by applying an [aggregate operations](https://www.postgresql.org/docs/current/functions-aggregate.html) when selecting the data. These *"aggregators"* take a collection of values and return a single value, such as the maximum or minimum values of a column. Note that when using aggregate functions, every column in the query that doesn't use an aggregate function needs to be in the GROUP BY clause.

In [15]:
# Query to return
# the columns average publication age and total record count
# from the time table
# grouped by day of the week
sql_query('''SELECT
                weekday,
                AVG(timedelta) as publication_age,
                COUNT(*) as weekday_count
              FROM time
              WHERE is_weekend = False
              GROUP by weekday;
              -- Aggregated columns are excluded from the GROUP BY
          ''')

weekday       publication_age  weekday_count
0     monday  351.6096682179852875           6661
1  wednesday  358.9706792199058507           7435
2    tuesday  351.9441136671177267           7390
3     friday  353.0396421680406946           5701
4   thursday  356.3575065363974130           7267

In addition to grouping and aggregating data from a single table, it is possible to [JOIN](https://www.postgresql.org/docs/11/queries-table-expressions.html) different tables from the same database, by matching ON specified columns. The two tables will then be joined when the value in each pair matches, returning rows with the data from both tables.

In [16]:
# Expansion of previous query to include the average shares from the news table
sql_query('''SELECT
                t.weekday,
                AVG(n.shares) as mean_shares,
                AVG(t.timedelta) as publication_age,
                COUNT(*) as weekday_count
              FROM time as t
              JOIN news as n
              ON t.ID = n.ID
              WHERE t.is_weekend = False
              GROUP by t.weekday;
              -- can use aliases instead of table names
          ''')


weekday            mean_shares       publication_age  weekday_count
0     monday  3647.0258219486563579  351.6096682179852875           6661
1  wednesday  3303.4053799596503026  358.9706792199058507           7435
2    tuesday  3202.5006765899864682  351.9441136671177267           7390
3     friday  3285.1810208735309595  353.0396421680406946           5701
4   thursday  3178.5991468281271501  356.3575065363974130           7267

SQL will perform an [INNER JOIN](https://www.postgresql.org/docs/11/tutorial-join.html) by default, that is, the rows returned by JOIN will have a successful match between the tables. Because rows are only returned when there is a match on both sides, it doesn't matter which table is on the left and which table is on the right. There are also three types of OUTER JOINS ( left, right and full). In a LEFT OUTER JOIN every row from the left table (the table named on the FROM clause) will be included in the output, even if there was no matching row on the right table. Rows without a match will be filled with NULL for the columns from the right table. Left outer joins are often called just LEFT JOINS. A RIGHT OUTER JOIN (or RIGHT JOIN) works similarly to a left join, except that all the rows from the right table are returned (the table defined on the JOIN clause), even if there is no match. Because the only difference is table order, left and right joins are equivalent if the table order is reversed. Lastly, a FULL OUTER JOIN (FULL JOIN or just OUTER JOIN) returns all records from both the left and right tables. Note that this can potentially return very large data sets, which can choke even a production database server.

Lastly, conditional clauses can be used when querying a database. The [CASE](https://www.postgresql.org/docs/11/plpgsql-control-structures.html#PLPGSQL-CONDITIONALS) keyword is used to set up conditions and then take action on the values of a column. For example, it is common to combine CASE statements with COUNT to do conditional counts. 

In [17]:
# Query to return
# the counts of existing records
# from the news table
# grouped by the popularity classification based on the share number
sql_query('''SELECT 
                (CASE WHEN shares > 1400 THEN 'popular' ELSE 'unpopular' END) popularity,
                COUNT(*) as counts
             FROM news
             GROUP BY popularity             
          ''')


popularity  counts
0  unpopular   20082
1    popular   19562

## Common table expressions 

One important concept to understand is that the result of every SQL query is in itself a table. That means joins can be used not just to join tables on existing tables, but also to join them on the results of other queries. One way to do that is to use common table expressions ([CTE's](https://www.postgresql.org/docs/11/queries-with.html)). There are two basic ways to use CTE's in SQL: step processing for queries (that are too complex for a single execution) or preprocessing to facilitate a join. The WITH statement will create another, intermediate table which can be used, for example, with JOIN clauses to aggregate data before joining tables (if JOIN is used on its own the aggregation happens after joining). 

In [18]:
# Example of aggregating on JOIN (occurs after joining)
sql_query('''SELECT
                channel.channel,
                time.weekday,
                AVG(time.timedelta) as time_week,
                COUNT(*) as total
            FROM time
            JOIN channel ON time.ID = channel.ID
            WHERE time.weekday LIKE 'thursday' OR time.weekday LIKE 'monday'
            GROUP BY channel.channel, time.weekday
            ORDER BY total DESC
            LIMIT 5
          ''')

channel   weekday             time_week  total
0          world  thursday  292.0280433397068196   1569
1  entertainment    monday  334.0000000000000000   1358
2          world    monday  286.6703539823008850   1356
3           tech  thursday  383.7977099236641221   1310
4           tech    monday  391.4688259109311741   1235

In [19]:
# Example of aggregating on CTE (occurs before joining)
sql_query('''-- Set up the CTE to create a weekday average share table.
            WITH
                time2
            AS (
                SELECT 
                    weekday,
                    AVG(timedelta) as time_week
                 FROM time
                 GROUP BY weekday
                )

            -- Joining the table created.
            SELECT
                channel.channel,
                time.weekday,
                time2.time_week,
                COUNT(*) as total
            FROM time
            -- Need to rejoin back with time to get the ID values
            JOIN time2 ON time.weekday = time2.weekday
            JOIN channel ON time.ID = channel.ID
            WHERE time.weekday LIKE 'thursday' OR time.weekday LIKE 'monday'
            GROUP BY channel.channel, time.weekday, time2.time_week
            ORDER BY total DESC
            LIMIT 5
          ''')


channel   weekday             time_week  total
0          world  thursday  356.3575065363974130   1569
1  entertainment    monday  351.6096682179852875   1358
2          world    monday  351.6096682179852875   1356
3           tech  thursday  356.3575065363974130   1310
4           tech    monday  351.6096682179852875   1235

Note how in the previous examples, the time_week averages are different. In the first case, the averages are only computed after the tables are joined, so it corresponds to the average for each channel/weekday combination. In the second case, the average time is estimated before the joining operation, thus it will be an unique value for each day of the week.

In [20]:
# Extension of previous popularity queries
# where the resulting table is used to aggregate popularity by weekday
sql_query('''-- Set up the CTE to create a weekday average share table.
            WITH
                popularity
            AS (
                SELECT 
                    (CASE WHEN shares > 1400 THEN 'popular' ELSE 'unpopular' END) popularity,
                    ID
                 FROM news
                )

            -- Joining the weekday table we created with the time to get IDs.
            SELECT
                time.weekday,
                popularity.popularity,
                COUNT(*) as total
            FROM popularity
            JOIN time
            ON
                popularity.ID =time.ID
            GROUP BY time.weekday, popularity.popularity
            ORDER BY total
          ''')


weekday popularity  total
0    saturday  unpopular    733
1      sunday  unpopular    976
2    saturday    popular   1720
3      sunday    popular   1761
4      friday  unpopular   2836
5      friday    popular   2865
6      monday    popular   3140
7   wednesday    popular   3335
8     tuesday    popular   3359
9    thursday    popular   3382
10     monday  unpopular   3521
11   thursday  unpopular   3885
12    tuesday  unpopular   4031
13  wednesday  unpopular   4100

The previous query exemplifies a simple analysis performed by querying a database to get informative data statistics. It is clear that the most common articles are unpopular articles, particularly those published from Monday to Thursday. Interestingly, this tendency for having more unpopular articles than popular articles is reversed Saturdays and Sundays, even though these days have a lower total article count in general. The articles published on Fridays are a special case, showing a balanced separation between popular and unpopular articles.

In [21]:
#### Session notes
%load_ext version_information
%version_information pandas, numpy, psycopg2, IPython

Software versions
Python 3.6.7 64bit [GCC 8.2.0]
IPython 6.5.0
OS Linux 4.15.0 44 generic x86_64 with Ubuntu 18.04 bionic
pandas 0.23.4
numpy 1.15.1
psycopg2 2.7.6.1 (dt dec pq3 ext lo64)
IPython 6.5.0
Thu Jan 31 15:30:17 2019 GMT

In [22]:
# Print PostgreSQL version
cursor.execute("SELECT version();")
record = cursor.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 9.6.10 on x86_64-pc-linux-gnu (Ubuntu 9.6.10-1.pgdg16.04+1), compiled by gcc (Ubuntu 5.4.0-6ubuntu1~16.04.10) 5.4.0 20160609, 64-bit',) 



In [23]:
# Connections to database must be closed
cursor.close()
connection.close()

In [24]:
# Query to close all connections to database
## select pg_terminate_backend(pid) from pg_stat_activity where datname='onlinenews'; ## 
# Useful if something goes wrong